In [1]:
import torch
from torchvision import transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import cv2
import os
import pandas as pd
from PIL import Image, ImageDraw

# Model path, images path, and CSV path
model_path = 'C:/Users/fahad/OneDrive - Oulun yliopisto/OULU/Thesis/camera_density/models/stanford_faster_rcnn.ckpt'
image_dir = 'C:/Users/fahad/OneDrive - Oulun yliopisto/OULU/Thesis/camera_density/Dataset/signals/images/'
csv_file_path = 'C:/Users/fahad/OneDrive - Oulun yliopisto/OULU/Thesis/camera_density/Dataset/signals/signals_image_metadata.csv'
save_dir = 'C:/Users/fahad/OneDrive - Oulun yliopisto/OULU/Thesis/camera_density/Dataset/crossings_annotated/'

# Load CSV for Image Metadata
df = pd.read_csv(csv_file_path)

# Preprocessing: Resize image to 640x640, convert to tensor, and normalize
transform = T.Compose([
    T.Resize((640, 640)),  # Resize to 640x640 as per the paper
    T.ToTensor(),  # Convert to tensor
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization based on ImageNet
])

# Load the Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=False)
model.load_state_dict(torch.load(model_path)['state_dict'])  # Load the pre-trained weights

# Set model to evaluation mode
model.eval()

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to run inference on a single image
def run_inference(image_path):
    image = Image.open(image_path).convert("RGB")  # Open the image using PIL
    image_tensor = transform(image).unsqueeze(0).to(device)  # Preprocess and add batch dimension

    # Perform inference
    with torch.no_grad():
        predictions = model(image_tensor)
    
    return predictions

# Function to draw bounding boxes on the image
def draw_boxes(image, boxes):
    draw = ImageDraw.Draw(image)
    for box in boxes:
        draw.rectangle(box.tolist(), outline="red", width=3)
    return image

# Perform inference on the dataset, save annotated images
for idx, row in df.iterrows():
    image_name = row['image_name']  # Assuming CSV contains 'image_name' column
    image_path = os.path.join(image_dir, image_name)

    # Run inference
    predictions = run_inference(image_path)

    # Process predictions (bounding boxes, confidence scores)
    boxes = predictions[0]['boxes'].cpu().numpy()  # Bounding boxes
    scores = predictions[0]['scores'].cpu().numpy()  # Confidence scores

    # Filter results based on a confidence threshold (e.g., 0.75)
    confidence_threshold = 0.75
    filtered_boxes = boxes[scores >= confidence_threshold]

    # Open the original image
    image = Image.open(image_path).convert("RGB")

    # Draw bounding boxes
    if len(filtered_boxes) > 0:
        annotated_image = draw_boxes(image, filtered_boxes)

        # Save the annotated image
        save_path = os.path.join(save_dir, f"annotated_{image_name}")
        annotated_image.save(save_path)
        print(f"Saved annotated image: {save_path}")
    else:
        print(f"No detections for {image_name}")



c:\Users\fahad\.conda\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\fahad\.conda\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\fahad/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 13.0MB/s]


RuntimeError: Attempting to deserialize object on CUDA device 1 but torch.cuda.device_count() is 1. Please use torch.load with map_location to map your storages to an existing device.